In [97]:
%pip install plotly geopandas pyshp shapely
# make sure the libraries are installed

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: plotly in c:\users\redhawc\.pyenv\pyenv-win\versions\3.11.2\lib\site-packages (5.17.0)




[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: c:\Users\redhawc\.pyenv\pyenv-win\versions\3.11.2\python.exe -m pip install --upgrade pip


In [98]:
import requests
import pandas as pd
import requests
import plotly.graph_objects as go
import warnings

warnings.filterwarnings(
    "ignore",
    "Converting to PeriodArray/Index representation will drop timezone information.",
)

In [99]:
# Constants
BUBBLE_SIZE_FACTOR = 2
DEFAULT_SEASON = "2021 Q3"
COLORSCALE_TEMPERATURE = "Reds"
COLOR_PRECIPITATION = "skyblue"
COLOR_WILDFIRE = "red"
TITLE_TEXT = "2021 US Wildfire Cases Visualisation<br>Compared with Temperature & Precipitation by State"
LAKE_COLOR = "rgb(255, 255, 255)"
ANNOTATION_TEXT = "The bigger the bubble, the greater the precipitation in this region."


def get_latest_events():

    try:

        response = requests.get("https://eonet.gsfc.nasa.gov/api/v3/events")

        if response.status_code == 200:

            return response.json()

        else:

            print(f"Error: {response.status_code}")

            return None

    except Exception as e:

        print(f"An error occurred: {e}")

        return None



def extract_country(event_title):
    # Extract country from event title
    if "States" in event_title:
        return "United States"
    return event_title.split(",")[-1].strip()


def preprocess_wildfires_df(events):
    warnings.filterwarnings(
        "ignore",
        "Converting to PeriodArray/Index representation will drop timezone information.",
    )
    # Check if events are present before preprocessing
    if not events:
        return pd.DataFrame()

    # Extract relevant information from each event
    data = [
        {
            "Event": event["title"],
            "Categories": event["categories"][0]["title"]
            if event["categories"]
            else None,
            "Date": event["geometry"][0]["date"] if event["geometry"] else None,
            "Location": event["geometry"][0]["coordinates"]
            if event["geometry"]
            else None,
        }
        for event in events["events"]
    ]
    df = pd.DataFrame(data)

    # Process 'Event' field to extract 'Country'
    df["Country"] = df["Event"].apply(lambda event: extract_country(event))

    df["Date"] = pd.to_datetime(df["Date"])
    df["Season"] = df["Date"].dt.to_period("Q").astype(str)
    df[["Longitude", "Latitude"]] = pd.DataFrame(
        df["Location"].tolist(), index=df.index
    )
    df = df[(df["Country"] == "United States") & (df["Categories"] == "Wildfires")]
    seasons = df["Season"].unique()
    new_seasons = {season: season[:4] + " " + season[4:] for season in seasons}
    df["Season"] = df["Season"].replace(new_seasons)

    return df


def get_temperature_and_precipitation_data():
    # data source: https://www.ncdc.noaa.gov/cag/statewide/time-series
    temperature = pd.read_csv(
        "https://raw.githubusercontent.com/REDhawC/EDI_BUSANA/master/Python_programming/coding_problems/TemperatureByStates_Updated.csv"
    )

    precipitation = pd.read_csv(
        "https://raw.githubusercontent.com/REDhawC/EDI_BUSANA/master/Python_programming/coding_problems/PrecipitationByStates_Updated.csv"
    )

    return (temperature, precipitation)


def create_choropleth_trace(data, season, name, colorbar_title):
    return go.Choropleth(
        visible=True,
        locations=data["code"],
        z=data[season].astype(float),
        locationmode="USA-states",
        colorscale=COLORSCALE_TEMPERATURE,
        name=name,
        autocolorscale=False,
        marker_line_color="white",
        colorbar_title=colorbar_title,
    )


def create_scattergeo_trace(data, season, name):
    COLOR_PRECIPITATION = "skyblue"
    BUBBLE_SIZE_FACTOR = 2

    return go.Scattergeo(
        visible=True,
        locations=data["code"],
        locationmode="USA-states",
        marker=dict(
            size=data[season] * BUBBLE_SIZE_FACTOR,
            color=COLOR_PRECIPITATION,
            opacity=1,
            line=dict(width=0),
        ),
        name=name,
    )


def create_wildfire_trace(df, season):
    COLOR_WILDFIRE = "red"
    DEFAULT_SEASON = "2021 Q3"

    season_df = df[df["Season"] == season]
    return go.Scattergeo(
        visible=(season == DEFAULT_SEASON),
        lon=season_df["Longitude"],
        lat=season_df["Latitude"],
        text=["🔥" for _ in range(len(season_df))],
        mode="text",
        name="Wildfire",
        textfont=dict(
            family="Arial",
            size=12,
            color=COLOR_WILDFIRE,
        ),
    )


def visualisation(temperature, precipitation, df):
    temperature["text"] = temperature["state"] + "<br>"
    fig = go.Figure()

    # Add temperature and precipitation traces
    fig.add_trace(
        create_choropleth_trace(
            temperature, DEFAULT_SEASON, "Temperature", "↑ Temperature (F)"
        )
    )
    fig.add_trace(
        create_scattergeo_trace(precipitation, DEFAULT_SEASON, "Precipitation")
    )

    # Add wildfire data traces for each quarter
    seasons = ["2021 Q1", "2021 Q2", DEFAULT_SEASON, "2021 Q4"]
    for season in seasons:
        fig.add_trace(create_wildfire_trace(df, season))

    fig.update_layout(
        title_text=TITLE_TEXT,
        geo=dict(
            scope="usa",
            projection=go.layout.geo.Projection(type="albers usa"),
            showlakes=True,
            lakecolor=LAKE_COLOR,
        ),
        showlegend=False,
    )

    # Add slider to switch quarters
    steps = []
    for index1, season in enumerate(seasons):
        step = dict(
            method="update",
            args=[
                {
                    "z": [temperature[season].astype(float)],
                    "marker.size": [precipitation[season] * BUBBLE_SIZE_FACTOR],
                    "visible": [True, True]
                    + [index1 == index2 for index2 in range(len(seasons))],
                }
            ],
            label=season,
        )
        steps.append(step)

    slider = dict(active=2, currentvalue={"prefix": "Season: "}, steps=steps)
    fig.update_layout(sliders=[slider])

    # Add annotation
    fig.add_annotation(
        x=0.55,
        y=-0.1,
        xref="paper",
        yref="paper",
        text=ANNOTATION_TEXT,
        showarrow=False,
        font=dict(size=12, color="purple"),
        align="center",
    )

    fig.show()

In [100]:
events = get_latest_events()
wildfire_df = preprocess_wildfires_df(events)
temperature, precipitation = get_temperature_and_precipitation_data()
wildfire_df.head()

,Event,Categories,Date,Location,Country,Season,Longitude,Latitude
22,"Major Fire, Ocala National Forest, Florida, Un...",Wildfires,2023-04-27 14:15:00+00:00,"[-81.688889, 29.229722]",United States,2023 Q2,-81.688889,29.229722
55,"Garceau Fire, W of Polson, Montana, United States",Wildfires,2022-08-16 00:00:00+00:00,"[-114.489, 47.694]",United States,2022 Q3,-114.489000,47.694000
105,"Wildfire - N of Merced, California - United St...",Wildfires,2022-02-16 20:44:00+00:00,"[-120.533005422, 37.543469882]",United States,2022 Q1,-120.533005,37.543470
106,"Wildfire - E of Mesa, Arizona - United States",Wildfires,2022-02-13 20:40:00+00:00,"[-110.862229905, 33.417091555]",United States,2022 Q1,-110.862230,33.417092
111,"Wildfire - E of Mesa, Arizona - United States",Wildfires,2022-01-28 08:38:00+00:00,"[-110.860071422, 33.416654251]",United States,2022 Q1,-110.860071,33.416654


In [101]:
temperature.head()
# check the temperature data

,State,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,2023 Q1,2023 Q2,2023 Q3,2023 Q4,code,state
0,Alabama,50.966667,69.333333,77.733333,58.766667,50.433333,72.133333,78.300000,55.266667,55.533333,70.133333,79.800000,65.2,AL,Alabama
1,Alaska,6.900000,38.366667,47.566667,13.100000,9.633333,39.233333,48.800000,17.500000,10.166667,35.466667,50.000000,27.8,AK,Alaska
2,Arizona,46.166667,69.900000,78.600000,53.766667,46.566667,69.366667,78.733333,50.166667,43.166667,65.766667,79.900000,64.6,AZ,Arizona
3,Arkansas,43.800000,67.366667,78.100000,56.100000,44.366667,69.633333,79.100000,51.500000,49.366667,69.200000,79.166667,64.1,AR,Arkansas
4,California,47.033333,65.866667,76.333333,52.166667,49.233333,63.433333,76.500000,51.266667,43.200000,61.333333,74.233333,62.1,CA,California


In [102]:
precipitation.head()
# check the precipitation data

,State,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,2023 Q1,2023 Q2,2023 Q3,2023 Q4,code,state
0,Alabama,5.330000,6.323333,6.583333,3.483333,5.880000,4.620000,4.783333,4.000000,5.556667,4.923333,3.876667,1.40,AL,Alabama
1,Alaska,2.623333,2.260000,4.353333,3.656667,3.463333,1.140000,5.653333,4.073333,2.593333,2.600000,4.526667,4.06,AK,Alaska
2,Arizona,0.756667,0.150000,2.430000,0.853333,0.426667,0.240000,2.320000,1.036667,1.790000,0.190000,1.186667,0.20,AZ,Arizona
3,Arkansas,4.556667,5.713333,3.203333,3.270000,4.520000,5.443333,2.956667,4.310000,6.276667,4.440000,3.593333,4.67,AR,Arkansas
4,California,2.256667,0.180000,0.250000,3.623333,0.580000,0.816667,0.453333,2.846667,6.286667,0.633333,0.466667,0.45,CA,California


In [103]:
visualisation(temperature, precipitation, wildfire_df)